In [29]:
import quandl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from authtoken import authtoken

In [ ]:
cpi = quandl.get("RATEINF/CPI_USA", authtoken=authtoken)
pall=quandl.get("LPPM/PALL")
plat=quandl.get("LPPM/PLAT")
rhod=quandl.get("JOHNMATT/RHOD")

In [ ]:
prices = pd.DataFrame({'pall': pall['USD AM'], 'plat':plat['USD AM'], 'rhod': rhod['New York 9:30'], 'cpi':cpi['Value']})
prices['cpi'] = prices['cpi'].fillna(method='ffill')
prices = prices.dropna()
prices['plat'] = prices['plat']/ prices['cpi']
prices['pall'] = prices['pall']/ prices['cpi']
prices['rhod'] = prices['rhod']/ prices['cpi']

names = ['plat', 'pall', 'rhod']
colors = ['r','g','b']

# Individual price analysis

To profit from the market, we must find a commodity that is undervalued or overvalued and whose price will correct quickly. 
Let's look at the inflation-ajusted price to compare valuations over long time scales. 

In [ ]:
axes = prices.loc[:,names].plot(subplots=True, figsize=(14,10), sharex=False, color=colors)
for name in ['pall','plat']:
    lm = linear_model.LinearRegression()
    X = np.array(range(len(prices.index.values))).reshape(-1, 1)
    y = np.log10(prices[name])
    lm.fit(X,y)
    pred = lm.predict(X)
    pred = 10**pred
    prices[name+'_reg'] = pred
    index = names.index(name)
    ax = axes[index]
    prices.loc[:,name+'_reg'].plot(ax=ax, color='k')
    ax.legend([name, 'regression'])

* Over the last 28 years, platinum prices varied (max/min ratio) by a factor 5, palladium by a factor 25, rhodium by a factor 50. We can expect to make money on rhodium and palladium, but hardly on platinum.
* Rhodium prices look stationary whereas palladium and platinum prices could be trendy (regression line in black). Hence, we will use different methods to assess how likely it is that a commodity is over or under-valued. 

# Price multiples

To determine if the price of commodity is stretched with respect to historic valuations, we plot the price ratio with respect to median price (rhodium) and regression price (platinum and palladium). 

In [ ]:
a = pd.DataFrame({'rhod': prices['rhod']/prices['rhod'].median(),
                  'pall': prices['pall']/prices['pall_reg'],
                  'plat': prices['plat']/prices['plat_reg']})

a.loc[:,names].plot(figsize=(14,5), color=colors)

* Rhodium: Current multiple (x6) suggests a bubble. But it could still go up and take years to burst like in 2006-2008. When the bubble bursts, expect prices to be divided by 6. 
* Palladium: Although historically high, price multiple are only half thoses of the 2001 bubble. 
* Platinum: Seems undervalued, but unfortunately platinum shows low volatility, especially to the upside. 

# price ratios

Palladium and platinum are mainly used in the auto industry to reduce toxic gas emissions. In theory car manufacturers could substitute one metal for another when prices ratios are too streched. Let's look at historic prices ratios between the two metals. 

In [ ]:
ratio = pd.DataFrame({'pall-plat':prices['pall']/prices['plat'],
                          'rhod-plat':prices['rhod']/prices['plat'],
                          'rhod-pall':prices['rhod']/prices['pall']
                         })
ratio.loc[:,'pall-plat'].plot(figsize=(15,5))
plt.title('palladium / platinum price ratio')
plt.figure()
ratio.loc[:,'rhod-plat'].plot(figsize=(15,5))
plt.title('rhodium / platinum price ratio')

* pall / plat ratio is at an all time high ! It has recently showed signed of volatility (end 2019). 
* The ratio has spent 90% of time below 1. How long can it maintain above 2 ? 
* The 2008 crisis hit both metals equally, the ratio didn't change at that time. 

# Price Variations

In [ ]:
daily = 100*prices.loc[:,names].pct_change(periods=1)
weekly = 100*prices.loc[:,names].pct_change(periods=7)
monthly = 100*prices.loc[:,names].pct_change(periods=30)
trimestre = 100*prices.loc[:,names].pct_change(periods=120)
year  = 100*prices.loc[:,names].pct_change(periods=360)

rplat = pd.DataFrame({'day':daily['plat'], 'week':weekly['plat'], 'month':monthly['plat'], 'trimestre':trimestre['plat'], 'year':year['plat']})
rpall = pd.DataFrame({'day':daily['pall'], 'week':weekly['pall'], 'month':monthly['pall'], 'trimestre':trimestre['pall'], 'year':year['pall']})
rrhod = pd.DataFrame({'day':daily['rhod'], 'week':weekly['rhod'], 'month':monthly['rhod'], 'trimestre':trimestre['rhod'], 'year':year['rhod']})

freqs = ['month', 'trimestre']
rpall.loc[:,freqs].rolling(15).mean().plot(figsize=(15,7))
rrhod.loc[:,freqs].rolling(15).mean().plot(figsize=(15,7))
#axes = year.plot(subplots=True, figsize=(15,10), color=colors)
#for ax in axes:
#    ax.axhline(y=0,color='black')

TODO

# Conclusion
Over the last 28 years, the three main components of the platinum group metals have shown rapid price bubbles build up and burst: palladium (%6 after the 2001 burst), platinum  and platinum (%8 and %2 respectively in 2008). 

Looking at multiples to historic prices and price ratios between elements of the metal groups, we showed that a third bubble is building for rhodium and palladium. 